# Automatic Detection and Classification of Rock Microstructures through Machine Learning

## Author(s)
List authors, their current affiliations,  up-to-date contact information, and ORCID if available. Add as many author lines as you need. 

- Author1 = {"name": "Stephen Iota", "affiliation": "USC", "email": "iota@usc.edu", "orcid": ""}
- Author2 = {"name": "Junyi Liu", "affiliation": "USC", "email": "liujunyi@usc.edu", "orcid": ""}
- Author3 = {"name": "Ming Lyu", "affiliation": "USC", "email": "minglyu@usc.edu", "orcid": ""}
- Author4 = {"name": "Bolong Pan", "affiliation": "USC", "email": "bolongpa@usc.edu", "orcid": ""}
- Author5 = {"name": "Xiaoyu Wang", "affiliation": "USC", "email": "xwang93@usc.edu", "orcid": ""}
- Author6 = {"name": "Yolanda Gil", "affiliation": "USC", "email": "gil@isi.edu", "orcid": ""}
- Author7 = {"name": "Wael AbdAlmageed", "affiliation": "USC", "email": "wamageed@isi.edu", "orcid": ""}
- Author8 = {"name": "Gurman Gill", "affiliation": "Sonoma State", "email": "gillg@sonoma.edu", "orcid": ""}
- Author9 = {"name": "Matty Mookerjee", "affiliation": "Sonoma State", "email": "matty.mookerjee@sonoma.edu", "orcid": ""}



<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Template-Notebook-for-EarthCube---Long-Version" data-toc-modified-id="Template-Notebook-for-EarthCube---Long-Version-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Template Notebook for EarthCube - Long Version</a></span><ul class="toc-item"><li><span><a href="#Author(s)" data-toc-modified-id="Author(s)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Author(s)</a></span></li><li><span><a href="#Purpose" data-toc-modified-id="Purpose-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Purpose</a></span></li><li><span><a href="#Technical-contributions" data-toc-modified-id="Technical-contributions-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Technical contributions</a></span></li><li><span><a href="#Methodology" data-toc-modified-id="Methodology-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Methodology</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Results</a></span></li><li><span><a href="#Funding" data-toc-modified-id="Funding-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Funding</a></span></li><li><span><a href="#Keywords" data-toc-modified-id="Keywords-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Keywords</a></span></li><li><span><a href="#Citation" data-toc-modified-id="Citation-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Citation</a></span></li><li><span><a href="#Work-In-Progress---improvements" data-toc-modified-id="Work-In-Progress---improvements-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Work In Progress - improvements</a></span></li><li><span><a href="#Suggested-next-steps" data-toc-modified-id="Suggested-next-steps-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Suggested next steps</a></span></li><li><span><a href="#Acknowledgements" data-toc-modified-id="Acknowledgements-1.11"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>Acknowledgements</a></span></li></ul></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Library-import" data-toc-modified-id="Library-import-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Library import</a></span></li><li><span><a href="#Local-library-import" data-toc-modified-id="Local-library-import-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Local library import</a></span></li></ul></li><li><span><a href="#Parameter-definitions" data-toc-modified-id="Parameter-definitions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Parameter definitions</a></span></li><li><span><a href="#Data-import" data-toc-modified-id="Data-import-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data import</a></span></li><li><span><a href="#Data-processing-and-analysis" data-toc-modified-id="Data-processing-and-analysis-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data processing and analysis</a></span><ul class="toc-item"><li><span><a href="#The-10-rules" data-toc-modified-id="The-10-rules-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>The 10 rules</a></span></li><li><span><a href="#Using-notebook-template" data-toc-modified-id="Using-notebook-template-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Using notebook template</a></span></li><li><span><a href="#Adding-table-of-contents" data-toc-modified-id="Adding-table-of-contents-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Adding table of contents</a></span></li><li><span><a href="#Creating-Binder-and-Docker-for-your-notebook-repository" data-toc-modified-id="Creating-Binder-and-Docker-for-your-notebook-repository-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Creating Binder and Docker for your notebook repository</a></span></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>References</a></span></li></ul></div>

## Purpose
Geologists need help classifying microscope rock images of sigma clasts; a type of mantled porphyroclasts widely used as kinematic indicators in rocks. 
Knowledge about the sheer sense of sigma clast during formation (either CCW or CW sheering) gives insights into rock formation history.  

This work reports on early investigation of machine learning techniques for automatic detection and classification of sigma clasts and their rotation from photomicrographs. 
Convolutional Neural Networks (CNNs) are used to extract and leverage defining features of sigma clasts, such as shape, color, texture, and tail direction to improve accuracy. 

We leverage existing models that are pre-trained on very large collections of images, and use transfer learning techniques to apply them to microstructure images.  

We used YOLOv3 to identify different sigma clasts in a given image. We also experimented with other large pre-trained models such as ResNet50, VGG19, Inceptionv3 with two additional layers trained specifically on our dataset. 

In order to facilitate exploration of different models with different settings, we are developing a computational experimentation environment to visualize different CNN network layers, classification heatmaps, and comparative metrics. 

Finally, since models perform better when more data are available, we are developing a web application to collect additional data from geoscientists and incentivize their participation in open science. 

The website allows researchers to upload images of rock microstructures, showing them the classification of the images based on the best models available, and allows them to correct any errors which can be used to improve the models. 

## Technical contributions
Summarize the central contributions of the notebook (libraries created, scientific analysis demonstrated, etc.), as a bulleted list, with one or more bullets.

Examples:

- demonstration of scientific analysis leveraging existing API (explain)
- development of underlying API that is exposed in the notebook (state which components are developed by the author, and point to documentation)
- developmnent of local libraries imported in the notebook (they should be subbmitted along with the notebook and documented in the same way)


## Methodology
Quickly describe assumptions and processing steps. Include URLs as necessary. 

## Results
Describe and comment on the most important results. Include images and URLs as necessary. 


## Funding
- No funding

## Keywords

keywords=["Sigma clast", "classification", "machine learning", "AI for geosciences", "CNNs for image classification"]

## Citation
S. Iota et. al, Automatic Detection and Classification of Rock Microstructures through Machine Learning, EarthCube Annual Meeting, 2021

## Work In Progress - improvements
Use this section only if the notebook is not final.

Notable TODOs:
- Data augmentation;
- Integrate image recognition models with user website;
- todo 3.

## Suggested next steps
State suggested next steps, based on results obtained in this notebook. This section is optional.


## Acknowledgements 

Include any relevant acknowledgements, apart from funding (which was in section 1.6)

This notebook template extends the original notebook template provided with the jupytemplate extension [5]. It is a result of collaboration between the TAC Working Group and the EarthCube Office. 

The template is licensed under a <a href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License.</a>

# Setup

## Library import
Import all the required Python libraries.

In [ ]:
## add any additional imports you needed down below, try to group them by their purpose

In [ ]:
# Std library 
import os
from pathlib import Path
from glob import glob

# Data manipulation
import pandas as pd
import numpy as np

# ML models
import keras
import cv2
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import ResNet50,VGG19
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

# Visualizations
import matplotlib.pyplot as plt

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

## Local library import
Import all the required local libraries.

In [ ]:
## None needed so far, but add them here if you need...

# Parameter definitions
Set all relevant parameters for the notebook. By convention, parameters are uppercase, while all the other variables follow Python's guidelines.


# Data import
Retrieve all the required data for the analysis. Include brief data descriptions, and DOIs where possible.

Please import data in one of two ways:
1. Reference data that is available online and continuously managed (preferred). 
2. If the data is relatively small, please include the files along with the notebook. If you deal with large data, include a subset.

In [ ]:
## here we should import all the data files,
## we need to make all data available locally (inside a Binder),
## so maybe we should only use a small subset of the data for demonstration purposes
## then give a pointer to where the rest of the data can be found on line
## (maybe thru a gdrive link)

# Data processing and analysis
The core of the notebook is here. Split this section into subsections as required, and explain processing and analysis steps.

Selected best practices for organizing and formatting your notebooks are included below. 

### Personal Classes and functions

In [1]:
class GradCAM:
    def __init__(self, model, classIdx, layerName=None):
        # store the model, the class index used to measure the class
        # activation map, and the layer to be used when visualizing
        # the class activation map
        self.model = model
        self.classIdx = classIdx
        self.layerName = layerName

        # if the layer name is None, attempt to automatically find
        # the target output layer
        if self.layerName is None:
            self.layerName = self.find_target_layer()

    def find_target_layer(self):
        # attempt to find the final convolutional layer in the network
        # by looping over the layers of the network in reverse order
        for layer in reversed(self.model.layers):
            # check to see if the layer has a 4D output
            if len(layer.output.shape) == 4:
                return layer.name

        # otherwise, we could not find a 4D layer so the GradCAM
        # algorithm cannot be applied
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM.")

    def compute_heatmap(self, image, eps=1e-8):
        # construct our gradient model by supplying (1) the inputs
        # to our pre-trained model, (2) the output of the (presumably)
        # final 4D layer in the network, and (3) the output of the
        # softmax activations from the model
        gradModel = Model(inputs=[self.model.inputs], outputs= [self.model.get_layer(self.layerName).output, self.model.output])

        # record operations for automatic differentiation
        with tf.GradientTape() as tape:
            # cast the image tensor to a float-32 data type, pass the
            # image through the gradient model, and grab the loss
            # associated with the specific class index
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            loss = predictions[:, self.classIdx]

        # use automatic differentiation to compute the gradients
        grads = tape.gradient(loss, convOutputs)

        # compute the guided gradients
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads

        # the convolution and guided gradients have a batch dimension
        # (which we don't need) so let's grab the volume itself and
        # discard the batch
        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]

        # compute the average of the gradient values, and using them
        # as weights, compute the ponderation of the filters with
        # respect to the weights
        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)

        # grab the spatial dimensions of the input image and resize
        # the output class activation map to match the input image
        # dimensions
        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))

        # normalize the heatmap such that all values lie in the range
        # [0, 1], scale the resulting values to the range [0, 255],
        # and then convert to an unsigned 8-bit integer
        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("uint8")

        # return the resulting heatmap to the calling function
        return heatmap

    def overlay_heatmap(self, heatmap, image, alpha=0.5,
        colormap=cv2.COLORMAP_JET):
        # apply the supplied color map to the heatmap and then
        # overlay the heatmap on the input image
        heatmap = cv2.applyColorMap(heatmap, colormap)
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)

        # return a 2-tuple of the color mapped heatmap and the output,
        # overlaid image
        return (heatmap, output)

NameError: name 'cv2' is not defined

In [ ]:
def get_img_array(img_path, size):
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    array = keras.preprocessing.image.img_to_array(img)
    # add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    print(grads.shape)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))#, 2))
    print(pooled_grads.shape)

    # multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
base_model=VGG19(weights='imagenet',include_top=False)
base_model.trainable = False

In [ ]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

In [ ]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory(folder_path,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])


step_size_train=train_generator.n//train_generator.batch_size
model.fit(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=50)

### Viz -- heatmaps and tensorboard

In [ ]:
# Generate class activation heatmap
heatmap = make_gradcam_heatmap(test_array, model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
%tensorboard --logdir logs



## The 10 rules
These recommendations are based on [6]

1. Tell a story for an audience: interleave explanatory text with code and results to create a computational narrative.

2. Document the process, not just the results: so that others, and yourself later, will understand your reasoning and choices.

3. Use cell divisions to make steps clear: have eaach cell perform one meaningful and documented step, avoid long cells.

4. Modularize code: use and document functions for repeated operations to make the code more readable and save space.

5. Record dependencies: use pip or Conda package manager, with environment.yml (Conda) or requirements.txt (pip). Please indicate package versions explicitly. Ideally, include a listing of dependencies at the bottom of the notebook. Make sure you test your notebook in an environment created from these dependencies, so that you don't add undocumened dependencies.

6. Use version control: https://github.com/jupyter/nbdime is a diffing tool specifically designed for notebooks; you may find it useful.

7. Build a pipeline: place key variable declarations at the top. Test with different parameters, clean up, and run all cells to prepare for potential non-interactive execution.

8. Share and explain your data: if the original data are too big, inlude a sample. Make sure that the data is accessible. Include data description, and any processing done beforehand. Ideally, point to datasets that have been permanently managed and identified by DOIs.

9. Design your notebooks to be read, run, and explored:  
    9.1. Read: store it in a public code repository with a clear README file and a liberal open source license. Generate HTML/PDF versions of the final notebook.  
    9.2. Run: use Binder to run it in the cloud, and/or create a Docker image.  
    9.3. Explore: use interactive widgets (e.g., ipywidgets, or panel) 
    
10. Advocate for open research: ask your colleagues or friends to try to run your notebooks.


__Additional recommendations, beyond the 10 rules:__
1. Do not include shell commands in cells (unless the notebook will be distributed in Docker container.)
2. Keep cell output if other researchers will use the included data, or clear cell output if they will use their own data.
3. Develop your code in a clean virtual environment, to avoid dependency conflicts and streamline packaging.

To create a virtual environment, use
```
conda create -n <enviroment_name> python=3 jupyter -y
```
or
```
python3 -m venv /path/to/new/virtual/environment
```
and then activate it. See more about creating virtual environments in [9], [10].

For additional useful guides see [1], [2], [3], [4]


Then replace the default template.ipynb with this notebook or with the short version (just headings and brief explanations).

Both the long and the short versions can be found at earthcube github.


To make the changes effective after replacing the template and renaming it to template.ipynb, run in terminal:
```
jupyter nbextension install --py jupytemplate --sys-prefix
jupyter nbextension enable jupytemplate/main --sys-prefix
```

## Creating Binder and Docker for your notebook repository

Sharing just the Jupyter notebook .ipynb file is not sufficient for broad re-usability. We strongly recommend that Jupyter notebooks are submitted to EarthCube in Binders. The Binder service is a free service that allows anyone to run notebooks from their web browser without installing any software and resolving dependencies.

Follow these steps:
 - Assemble the Jupyter notebooks , environment files, custom external libraries, datasets, and a README, in a single project directory (and sub-directories off of it, as needed). Use GitHub to organize your project as a GitHub repository.


 - Create Binder. Use http://mybinder.org to create a  URL for your notebook Binder (you will need to enter your GitHub repo URL). You can also add a Launch Binder button directly to your GitHub repo, by including the following in your README.md:

```
launch with myBinder
[![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/<path to your repo>)
```

Specific projects may have their own Binder implementations. In this case, include a project-specific Binder reference so that reviewers can execute your code.

 - Optionally, test your project with repo2docker. See instructions in [2]. This will create a docker image which can be made available in a Docker repository. 


Expect the reviewers to use a browser of their choice to run your code, and try to avoid special execution requirements that make it difficult for users to appreciate your work.


# References
List relevant references.

1. Notebook sharing guidelines from reproducible-science-curriculum: https://reproducible-science-curriculum.github.io/publication-RR-Jupyter/
2. Guide for developing shareable notebooks by Kevin Coakley, SDSC: https://github.com/kevincoakley/sharing-jupyter-notebooks/raw/master/Jupyter-Notebooks-Sharing-Recommendations.pdf
3. Guide for sharing notebooks by Andrea Zonca, SDSC: https://zonca.dev/2020/09/how-to-share-jupyter-notebooks.html
4. Jupyter Notebook Best Practices: https://towardsdatascience.com/jupyter-notebook-best-practices-f430a6ba8c69
5. Introduction to Jupyter templates nbextension: https://towardsdatascience.com/stop-copy-pasting-notebooks-embrace-jupyter-templates-6bd7b6c00b94  
    5.1. Table of Contents (Toc2) readthedocs: https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/toc2/README.html  
    5.2. Steps to install toc2: https://stackoverflow.com/questions/23435723/installing-ipython-notebook-table-of-contents
6. Rule A, Birmingham A, Zuniga C, Altintas I, Huang SC, et al. (2019) Ten simple rules for writing and sharing computational analyses in Jupyter Notebooks. PLOS Computational Biology 15(7): e1007007. https://doi.org/10.1371/journal.pcbi.1007007. Supplementary materials: example notebooks (https://github.com/jupyter-guide/ten-rules-jupyter) and tutorial (https://github.com/ISMB-ECCB-2019-Tutorial-AM4/reproducible-computational-workflows)
7. Languages supported by Jupyter kernels: https://github.com/jupyter/jupyter/wiki/Jupyter-kernels
8. EarthCube notebooks presented at EC Annual Meeting 2020: https://www.earthcube.org/notebooks
9. Manage your Python Virtual Environment with Conda: https://towardsdatascience.com/manage-your-python-virtual-environment-with-conda-a0d2934d5195
10. Venv - Creation of Virtual Environments: https://docs.python.org/3/library/venv.html